In [41]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
from datasets import GeonamesDataset
import polars as pl

In [43]:
geonames = GeonamesDataset("./data/cities500.txt.gz", max_len=14)

In [44]:
geonames.df.sample(10)

sequence,feature code,country code,population
str,str,str,i64
"""Toutey""","""PPL""","""CI""",996
"""Barbouche""","""PPL""","""DZ""",4897
"""Mulbach""","""PPLA4""","""DE""",125
"""Capendu""","""PPL""","""FR""",1525
"""Cantarana""","""PPLA3""","""IT""",257
"""Bhusaval""","""PPL""","""IN""",183001
"""Ciguha""","""PPLA4""","""ID""",0
"""Rathkeale""","""PPL""","""IE""",1550
"""Geisenhausen""","""PPL""","""DE""",6367


In [45]:
df = geonames.df
alphabet = "".join(
    set("".join(df.get_column("sequence").str.split("").explode().to_list()))
)

In [46]:
from utils import Tokenizer

t = Tokenizer(
    alphabet=alphabet,
    max_len=16,
)

In [47]:
X = t.encode(df)

In [48]:
import torch

total_samples = X.size(0)

# Define the proportions for train, test, and validation sets
train_ratio = 0.8
test_ratio = 0.1
val_ratio = 0.1

# Calculate the number of samples for each set
num_train = int(total_samples * train_ratio)
num_test = int(total_samples * test_ratio)
num_val = total_samples - num_train - num_test

# Generate random indices
indices = torch.randperm(total_samples)

# Split the indices into train, test, and validation sets
train_indices = indices[:num_train]
test_indices = indices[num_train : num_train + num_test]
val_indices = indices[num_train + num_test :]

# Create the train, test, and validation sets
X_train = X[train_indices]
X_test = X[test_indices]
X_val = X[val_indices]

print("Train set size:", X_train.shape)
print("Test set size:", X_test.shape)
print("Validation set size:", X_val.shape)

Train set size: torch.Size([101200, 16])
Test set size: torch.Size([12650, 16])
Validation set size: torch.Size([12651, 16])


In [49]:
from torch.utils.data import TensorDataset, DataLoader

train = TensorDataset(X_train)
test = TensorDataset(X_test)
val = TensorDataset(X_val)

In [50]:
for i, batch in enumerate(DataLoader(train, batch_size=64)):
    break

In [51]:
x = batch[0].float()

In [160]:
from torch import nn

In [246]:
import math


class PositionalEncoding(nn.Module):
    def __init__(
        self,
        d_model: int,
        d_output: int,
        dropout: float = 0.1,
        max_len: int = 5000,
    ):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        self.d_output = d_output
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model)
        )
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer("pe", pe)

    def forward(self, t: torch.Tensor) -> torch.Tensor:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = self.pe[t].squeeze(-1)
        return self.dropout(x)

In [247]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_normal_(m.weight)


class MLPWithTime(nn.Module):
    def __init__(
        self,
        token_embed: nn.Module,
        time_embed: nn.Module,
        hidden: nn.Module,
        output: nn.Module,
        alphabet_size: int,
        n_tokens: int,
    ):
        super().__init__()
        self.token_embed: nn.Module = token_embed
        self.time_embed: nn.Module = time_embed
        self.hidden: nn.Module = hidden
        self.output: nn.Module = output
        self.alphabet_size = alphabet_size
        self.n_tokens = n_tokens
        self.apply(init_weights)

    def forward(self, x: torch.Tensor, t: torch.Tensor) -> torch.Tensor:
        te = self.token_embed(x)
        pe = self.time_embed(t)
        he = self.hidden(te + pe)
        output = self.output(he)
        return output.reshape(
            -1,
            self.alphabet_size,
            self.n_tokens,
        )

In [248]:
token_dim = 16
word_embeding_dim = time_embeding_dim = 256
intermediate_dim = 128
hidden_dim = 64
dropout = 0.3
max_T = 1000
alphabet_size = len(t.stoi)

Input = nn.Sequential(
    nn.Linear(token_dim, word_embeding_dim),
    nn.BatchNorm1d(word_embeding_dim),
    nn.ReLU(),
    nn.Dropout(dropout),
)
TimeInput = PositionalEncoding(
    d_model=1,
    d_output=time_embeding_dim,
    max_len=max_T,
)
Hidden = nn.Sequential(
    nn.Linear(word_embeding_dim, intermediate_dim),
    nn.BatchNorm1d(intermediate_dim),
    nn.ReLU(),
    nn.Dropout(dropout),
    # ----
    nn.Linear(intermediate_dim, hidden_dim),
    nn.BatchNorm1d(hidden_dim),
    nn.ReLU(),
    nn.Dropout(dropout),
    # ----
    nn.Linear(hidden_dim, hidden_dim),
    nn.BatchNorm1d(hidden_dim),
    nn.ReLU(),
    nn.Dropout(dropout),
    # ----
    nn.Linear(hidden_dim, hidden_dim),
    nn.BatchNorm1d(hidden_dim),
    nn.ReLU(),
    nn.Dropout(dropout),
    # ----
    nn.Linear(hidden_dim, intermediate_dim),
    nn.BatchNorm1d(intermediate_dim),
    nn.ReLU(),
    nn.Dropout(dropout),
    # ----
    nn.Linear(intermediate_dim, intermediate_dim),
    nn.BatchNorm1d(intermediate_dim),
    nn.ReLU(),
    nn.Dropout(dropout),
    # ----
    nn.Linear(intermediate_dim, intermediate_dim),
    nn.BatchNorm1d(intermediate_dim),
    nn.ReLU(),
    nn.Dropout(dropout),
    # ----
)
Output = nn.Sequential(
    nn.Linear(intermediate_dim, alphabet_size * token_dim),
    nn.BatchNorm1d(alphabet_size * token_dim),
    nn.ReLU(),
    nn.Dropout(dropout),
)

model = MLPWithTime(
    time_embed=TimeInput,
    token_embed=Input,
    hidden=Hidden,
    output=Output,
    alphabet_size=alphabet_size,
    n_tokens=token_dim,
)

In [249]:
time = torch.randint(0, max_T, size=(x.shape[0],))

In [250]:
x.shape

torch.Size([64, 16])

In [251]:
x.shape

torch.Size([64, 16])

In [252]:
with torch.no_grad():
    print(model(x, time).shape)

torch.Size([64, 55, 16])


In [253]:
sum(p.numel() for p in model.parameters())

212368

# Training loop

In [241]:
from diffusion import ForwardDiffusionProcess, Scheduler

In [242]:
scheduler = Scheduler(T=max_T)
diffusion = ForwardDiffusionProcess(scheduler=scheduler)

In [244]:
import torch.nn.functional as F
from torch.optim import AdamW
from tqdm.notebook import tqdm
import torch.optim.lr_scheduler as lr_scheduler

batch_size = 4096
# num_epochs = 2000
# scheduler = lr_scheduler.CosineAnnealingLR(
# optimizer,
# T_max=40,
# )

for lr in torch.logspace(-4, 1, 100):
    optimizer = AdamW(model.parameters(), lr=lr.item())
    # training mode
    model.train()
    losses = []
    for batch_num in range(X_train.shape[0] // batch_size):
        optimizer.zero_grad()

        ix = torch.randint(0, X_train.shape[0], size=(batch_size,))
        batch = X_train[ix].long()
        time = torch.randint(low=0, high=max_T, size=(batch.shape[0],))

        x_0 = batch.float()
        x_t = diffusion.sample_T(x_0, time)
        x_0_pred = model(x_t, time)

        loss = F.cross_entropy(x_0_pred, batch)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()

    # validation mode
    model.eval()
    with torch.no_grad():
        time = torch.randint(0, max_T, size=(X_test.shape[0],))
        x_0_test = X_test.float()
        x_t_test = diffusion.sample_T(x_0_test, time)
        x_0_test_pred = model(x_t_test, time)
        test_loss = F.cross_entropy(x_0_test_pred, X_test.long())

        print(
            f"{lr=}, train_loss={torch.tensor(losses).mean().item():2.4f}, val_loss={test_loss.item():2.4f}"
        )

lr=tensor(1.0000e-04), train_loss=4.4510, val_loss=4.0677
lr=tensor(0.0001), train_loss=4.4044, val_loss=4.0973
lr=tensor(0.0001), train_loss=4.3310, val_loss=4.0879
lr=tensor(0.0001), train_loss=4.2429, val_loss=4.0449
lr=tensor(0.0002), train_loss=4.1603, val_loss=3.9875
lr=tensor(0.0002), train_loss=4.0926, val_loss=3.9216
lr=tensor(0.0002), train_loss=4.0293, val_loss=3.8384
lr=tensor(0.0002), train_loss=3.9756, val_loss=3.7355
lr=tensor(0.0003), train_loss=3.9302, val_loss=3.6811
lr=tensor(0.0003), train_loss=3.8918, val_loss=3.6277
lr=tensor(0.0003), train_loss=3.8517, val_loss=3.5701
lr=tensor(0.0004), train_loss=3.8031, val_loss=3.5599
lr=tensor(0.0004), train_loss=3.7571, val_loss=3.5322
lr=tensor(0.0005), train_loss=3.7066, val_loss=3.4856
lr=tensor(0.0005), train_loss=3.6614, val_loss=3.4833
lr=tensor(0.0006), train_loss=3.6277, val_loss=3.4879
lr=tensor(0.0006), train_loss=3.6071, val_loss=3.5061
lr=tensor(0.0007), train_loss=3.5893, val_loss=3.4919
lr=tensor(0.0008), train

In [254]:
import torch.nn.functional as F
from torch.optim import AdamW
from tqdm.notebook import tqdm
import torch.optim.lr_scheduler as lr_scheduler

batch_size = 4096
num_epochs = 2000
optimizer = AdamW(model.parameters(), lr=0.1)

scheduler = lr_scheduler.ExponentialLR(
    optimizer,
    gamma=0.99,
)


for epoch in tqdm(range(num_epochs)):
    # training mode
    model.train()
    losses = []
    for batch_num in range(X_train.shape[0] // batch_size):
        optimizer.zero_grad()

        ix = torch.randint(0, X_train.shape[0], size=(batch_size,))
        batch = X_train[ix].long()
        time = torch.randint(low=0, high=max_T, size=(batch.shape[0],))

        x_0 = batch.float()
        x_t = diffusion.sample_T(x_0, time)
        x_0_pred = model(x_t, time)

        loss = F.cross_entropy(x_0_pred, batch)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
        scheduler.step()

    # validation mode
    model.eval()
    with torch.no_grad():
        time = torch.randint(0, max_T, size=(X_test.shape[0],))
        x_0_test = X_test.float()
        x_t_test = diffusion.sample_T(x_0_test, time)
        x_0_test_pred = model(x_t_test, time)
        test_loss = F.cross_entropy(x_0_test_pred, X_test.long())

        print(
            f"{epoch=}, train_loss={torch.tensor(losses).mean().item():2.4f}, val_loss={test_loss.item():2.4f}"
        )

  0%|          | 0/2000 [00:00<?, ?it/s]

epoch=0, train_loss=3.3057, val_loss=2.6303
epoch=1, train_loss=2.7198, val_loss=2.2339
epoch=2, train_loss=2.6471, val_loss=2.1931
epoch=3, train_loss=2.6134, val_loss=2.1676
epoch=4, train_loss=2.5984, val_loss=2.1324
epoch=5, train_loss=2.5879, val_loss=2.1117
epoch=6, train_loss=2.5809, val_loss=2.0998
epoch=7, train_loss=2.5743, val_loss=2.0930
epoch=8, train_loss=2.5719, val_loss=2.0774
epoch=9, train_loss=2.5667, val_loss=2.0714
epoch=10, train_loss=2.5682, val_loss=2.0659
epoch=11, train_loss=2.5659, val_loss=2.0648
epoch=12, train_loss=2.5639, val_loss=2.0605
epoch=13, train_loss=2.5632, val_loss=2.0585
epoch=14, train_loss=2.5611, val_loss=2.0566
epoch=15, train_loss=2.5639, val_loss=2.0545
epoch=16, train_loss=2.5615, val_loss=2.0517
epoch=17, train_loss=2.5610, val_loss=2.0532
epoch=18, train_loss=2.5617, val_loss=2.0517
epoch=19, train_loss=2.5586, val_loss=2.0489
epoch=20, train_loss=2.5605, val_loss=2.0496
epoch=21, train_loss=2.5598, val_loss=2.0518
epoch=22, train_loss

KeyboardInterrupt: 

In [255]:
from diffusion import ForwardDiffusionProcess
scheduler = Scheduler(T=max_T)
diffusion = ForwardDiffusionProcess(scheduler=scheduler)

In [261]:
denoised = diffusion.sample(model=model, n=10, max_T=max_T)

In [262]:
t.decode_raw(denoised.argmax(1))

['<Soelhreein><>><',
 '<Saneeeaaea>>...',
 '<Sanaeeaaea>>...',
 '<Woelhreein><>><',
 '<Soelhreein><>><',
 '<Soelhreein><>><',
 '<Soelhreein><>><',
 '<Saneeeaaea>>...',
 '<Saneeeaaea>>...',
 '<Sanaenaae>>....']